- (re)train model (y)
- make new model (with web service input and output modules?)
- where to get experiment/model info (if neccesary):
    - 	 "Click on Use Web Service to go to the Consume tab in the Menu."
	- "Copy the values of the Primary Key and the Request-Response fields and save them in a document."